In [1]:
library(Signac)
library(Seurat)
library(EnsDb.Hsapiens.v86)
library(SeuratDisk)
library(dplyr)
library(BSgenome.Hsapiens.UCSC.hg38)

Warning message:
“package ‘Signac’ was built under R version 4.3.3”
Warning message:
“package ‘Seurat’ was built under R version 4.3.3”
Loading required package: SeuratObject

Warning message:
“package ‘SeuratObject’ was built under R version 4.3.3”
Loading required package: sp

Warning message:
“package ‘sp’ was built under R version 4.3.3”

Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: ensembldb

Warning message:
“package ‘ensembldb’ was built under R version 4.3.2”
Loading required package: BiocGenerics

Warning message:
“package ‘BiocGenerics’ was built under R version 4.3.2”

Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
 

In [ ]:
ifnb <- readRDS("hc_processed_rna+atac.rds")

In [3]:
ifnb <- subset(ifnb, celltype == "inhibitory neurons")

In [4]:
DefaultAssay(ifnb) <- "ATAC"
# first compute the GC content for each peak
ifnb <- RegionStats(ifnb, genome = BSgenome.Hsapiens.UCSC.hg38)

Warning message in RegionStats.default(object = regions, genome = genome, verbose = verbose, :
"Not all seqlevels present in supplied genome"


In [ ]:
idents.plot <- c('inhibitory neurons')
Idents(ifnb) <- idents.plot

In [7]:
ifnb

An object of class Seurat 
333440 features across 7739 samples within 3 assays 
Active assay: ATAC (265980 features, 247295 variable features)
 2 layers present: counts, data
 2 other assays present: RNA, SCT
 2 dimensional reductions calculated: pca, lsi

### Helper

In [6]:
library(ggplot2)
save_link_counts <- function(ifnb, gene_list, file_prefix){
    # link peaks to genes
    ifnb <- LinkPeaks(
      object = ifnb,
      peak.assay = "ATAC",
      expression.assay = "SCT",
      genes.use = gene_list
    )

    peak_gene_links <- Links(ifnb[["ATAC"]])
    df <- as.data.frame(peak_gene_links)

    gene_link_counts <- df %>%
      filter(pvalue < 0.05) %>%
      group_by(gene) %>%
      summarise(link_count = n())

    write.csv(gene_link_counts, paste0(file_prefix, "gene_link_counts.csv"), row.names = FALSE)
    
    # Return both the original df and gene_link_counts as a list
    return(list(df = df, gene_link_counts = gene_link_counts))
}

Warning message:
"package 'ggplot2' was built under R version 4.3.3"


In [7]:
library(Signac)
library(patchwork)   # lets us manipulate the assembled plot
library(ggplot2)

plot_coverage <- function(ifnb, gene_list, file_prefix,
                          idents.plot = NULL,
                          extend.up = 5e4, extend.down = 5e5) {

  DefaultAssay(ifnb) <- "ATAC"
  ifnb <- LinkPeaks(ifnb,
                    peak.assay       = "ATAC",
                    expression.assay = "SCT",
                    genes.use        = gene_list)

  for (g in gene_list) {
    message("Plotting ", g)

    tryCatch({

      cov <- CoveragePlot(
             ifnb,
             region            = g,
             features          = g,
             expression.assay  = "SCT",
             links             = FALSE,
             link.assay        = "ATAC",
             idents            = idents.plot,
             extend.upstream   = extend.up,
             extend.downstream = extend.down
           )
       arc  <- LinkPlot(ifnb, region = g, assay = "ATAC",
                  extend.upstream = 5e4, extend.downstream = 5e5) +
       scale_colour_gradient2(low = "blue", mid = "grey",
                               high = "#ca0020", midpoint = 0,
                               limits = c(-0.1, 0.1), name = "Link\nscore") +
        guides(colour = guide_colourbar(title.position = "top")) +
        theme(legend.position = "right")

       final <- cov / arc

       ggsave(paste0(file_prefix, g, ".pdf"), final, width = 15, height = 6)

    }, error = function(e) {
      message("Skipping ", g, ": ", e$message)
    })
  }
}

Warning message:
"package 'patchwork' was built under R version 4.3.3"


### Marker Genes

In [ ]:
library(readxl)
library(dplyr)

top_genes <- c()
# Define file path
file_path <- "../../Annotate_Inh_Subtype/Inh_subset_cells/Marker_Genes.xlsx"

sheet_names <- excel_sheets(file_path)
all_sheets <- lapply(sheet_names, function(sheet) {
  read_excel(file_path, sheet = sheet)
})
names(all_sheets) <- sheet_names

for (name in sheet_names){
    print(name)
    gene_data <- all_sheets[[name]]
    # Ensure column names are correctly recognized
    colnames(gene_data) <- c("gene", "p_val", "avg_log2FC", "pct.1", "pct.2", "p_val_adj")

    # Filter genes based on criteria: p_val_adj < 0.001 and avg_log2FC > 0.25
    filtered_genes <- gene_data %>%
      filter(p_val_adj < 0.001, avg_log2FC > 2)
    
    top_genes <- c(top_genes, as.vector(filtered_genes$gene))
}

Warning message:
"package 'readxl' was built under R version 4.3.3"
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`
New names:
• `` -> `...1`


[1] "Inh L1 SST CXCL14"
[1] "Inh L1-4 LAMP5 DUSP4"
[1] "Inh L3-5 SST MAFB"
[1] "Inh L1 PAX6 CA4"
[1] "Inh L2-4 PVALB C8orf4"
[1] "Inh L5-6 PVALB STON2"
[1] "Inh L1-6 LAMP5 CA13"
[1] "Inh L6 LAMP5 ANKRD20A11P"
[1] "Inh L2-4 SST AHR"
[1] "Inh L5 PVALB CNTNAP3P2"
[1] "Inh L1 LAMP5 NDNF"
[1] "Inh L5-6 PVALB FAM150B"
[1] "Inh L1-3 VIP SSTR1"
[1] "Inh L1 ADARB2 ADAM33"
[1] "Inh L4-6 SST MTHFD2P6"
[1] "Inh L5-6 LAMP5 SFTA3"
[1] "Inh L1-3 VIP GGH"
[1] "Inh L1-3 PVALB WFDC2"
[1] "Inh L5-6 SST ISOC1"
[1] "Inh L5-6 SST TH"
[1] "Inh L5-6 SST KLHL14"
[1] "Inh L1 LAMP5 GGT8P"


In [9]:
length(top_genes)

[1] 3810

In [ ]:
file_prefix <- "../../Signac_plots/HC/Marker_Genes_Inh/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

omit <- save_link_counts(ifnb, top_genes, file_prefix)

Directory created: ../Signac_plots/HC/Marker_Genes_Inh/

Testing 1619 genes and 138047 peaks

Found gene coordinates for 1162 genes

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270721.1, KI270726.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, GL000213.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


In [ ]:
file_prefix <- "../../Signac_plots/HC/Marker_Genes_Inh/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, top_genes, file_prefix)

Directory already exists: ../Signac_plots/HC/Marker_Genes_RORB_exn/

Testing 577 genes and 148617 peaks

Found gene coordinates for 354 genes

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270721.1, KI270726.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, GL000213.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."
Plotting TLL1

Plotting GRIN3A

Warning message:
"Removed 7 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Plotting COL21A1

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Warning message:
"Removed 3 rows containing missing values or values outside the scale range
(`geom_segment()`)."
Plo

### DEG RORB (both RBD vs HC and PD vs HC)

In [ ]:
deg = read.csv("../../Signac_plots/PD/DEGs_PD_vs_HC_Inh/gene_link_counts.csv")$gene

In [ ]:
file_prefix <- "../../Signac_plots/HC/DEGs_RBD_PD_vs_HC_Inh/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

save_link_counts(ifnb, deg, file_prefix)

Directory already exists: ../Signac_plots/HC/DEGs_RBD_PD_vs_HC_Inh/

Testing 161 genes and 138047 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270721.1, KI270726.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, GL000213.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."


In [ ]:
file_prefix <- "../../Signac_plots/HC/DEGs_RBD_PD_vs_HC_Inh/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

plot_coverage(ifnb, deg, file_prefix)

### Save individually for viz

In [ ]:
file_prefix <- "../../Signac_plots/HC/DEGs_RBD_PD_vs_HC_Inh/"

# Create the directory if it doesn't exist
if (!dir.exists(file_prefix)) {
  dir.create(file_prefix, recursive = TRUE)  # 'recursive = TRUE' ensures parent directories are created if needed
  message(paste("Directory created:", file_prefix))
} else {
  message(paste("Directory already exists:", file_prefix))
}

one <- c("LDB2")
plot_coverage(ifnb, one, file_prefix)

Directory already exists: ../Signac_plots/HC/DEGs_RBD_PD_vs_HC_Inh/

Testing 1 genes and 138047 peaks

Warning message in .merge_two_Seqinfo_objects(x, y):
"Each of the 2 combined objects has sequence levels not in the other:
  - in 'x': GL000194.1, GL000195.1, GL000205.2, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270721.1, KI270726.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1, GL000009.2, GL000213.1
  - in 'y': chrMT
  Make sure to always combine/compare objects based on the same reference
  genome (use suppressWarnings() to suppress this warning)."
Plotting LDB2

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
